In [102]:

# Ocean Wave Height and Period Forecasting with DeepAR
# Deep Autoregressive Time Series Modeling using PyTorch Forecasting

import warnings, numpy as np, pandas as pd, torch
import matplotlib.pyplot as plt
import lightning as pl
import pytorch_forecasting as ptf
from pytorch_forecasting import TimeSeriesDataSet
from sktime.split import temporal_train_test_split
import importlib


from oceanwave_forecast import data_manager, data_pipeline, forecasting_utils, config, mlflow_utils, training, plotting

importlib.reload(data_manager)
importlib.reload(data_pipeline)
importlib.reload(forecasting_utils)
importlib.reload(config)
importlib.reload(mlflow_utils)
importlib.reload(training)
importlib.reload(plotting)

from collections import namedtuple
from pytorch_forecasting import TimeSeriesDataSet
from pytorch_forecasting.data.encoders import GroupNormalizer, MultiNormalizer
from sktime.transformations.series.summarize import WindowSummarizer

from pytorch_forecasting import DeepAR
from lightning.pytorch import Trainer
from lightning.pytorch.callbacks import EarlyStopping, LearningRateMonitor
from pytorch_forecasting import MultiLoss, NormalDistributionLoss


from loguru import logger

# Set random seeds for reproducibility
pl.seed_everything(config.RANDOM_STATE)
torch.manual_seed(config.RANDOM_STATE)
np.random.seed(config.RANDOM_STATE)

import mlflow
from mlflow.exceptions import MlflowException
from mlflow.tracking import MlflowClient
from urllib.parse import quote


from pytorch_forecasting.metrics import MAE, RMSE, SMAPE, MAPE

2025-07-28 01:41:41.715 | INFO     | oceanwave_forecast.config:<module>:12 - PROJ_ROOT path is: D:\CML\Term 8\ML projects\forecasting_workspace\oceanwave_forecast
Global seed set to 42


# 1. DATA PREPARATION AND PREPROCESSING


In [87]:
# CHecking the experiment and its runs:

def fetch_and_log_runs_structured(experiment_name: str):
    """Fetch and display MLflow runs organized by run_number, using PROJ_ROOT/mlruns."""
    # ——— 1. ensure tracking URI is set to PROJ_ROOT/mlruns ———
    mlruns_path = config.PROJ_ROOT / "mlruns"
    mlruns_path.mkdir(parents=True, exist_ok=True)
    # URL‑encode to handle spaces, windows paths, etc.
    uri = f"file:///{quote(str(mlruns_path.absolute()), safe=':/')}"
    mlflow.set_tracking_uri(uri)
    logger.info(f"MLflow tracking URI set to: {uri}")

    try:
        # ——— 2. locate experiment by name ———
        client = MlflowClient()
        exp = client.get_experiment_by_name(experiment_name)
        if exp is None:
            logger.error(f"Experiment '{experiment_name}' not found at {mlruns_path}.")
            return

        # ——— 3. pull runs as DataFrame ———
        runs_df = mlflow.search_runs(
            experiment_ids=[exp.experiment_id],
            order_by=["attributes.start_time DESC"],
            output_format="pandas",
        )
        if runs_df.empty:
            logger.warning(f"No runs found for experiment '{experiment_name}'.")
            return

        # ——— 4. assemble detailed run info ———
        runs_data = []
        for rid in runs_df["run_id"]:
            run = client.get_run(rid)
            runs_data.append({
                "run_id": rid,
                "params": run.data.params,
                "metrics": run.data.metrics,
            })
        # sort by the integer run_number param
        runs_data.sort(key=lambda x: int(x["params"]["run_number"]))

        # ——— 5. pretty‑print ———
        print("=" * 60)
        print("MLFLOW EXPERIMENT RUNS")
        print("=" * 60)
        for run_data in runs_data:
            params = run_data["params"]
            metrics = run_data["metrics"]
            run_number = params["run_number"]
            model_name = params.get("model_type", params.get("model_name", "Unknown"))

            print(f"\n🔹 RUN {run_number} - {model_name}")
            print("-" * 40)
            print("📋 PARAMETERS:")
            for k, v in params.items():
                if k not in {"run_number", "model_type", "model_name"}:
                    print(f"  • {k}: {v}")
            print("📊 METRICS:")
            if metrics:
                for mk, mv in metrics.items():
                    print(f"  • {mk}: {mv:.6f}")
            else:
                print("  • No metrics recorded")
            print()

    except MlflowException as e:
        logger.exception(f"MLflow error: {e}")
    except Exception as e:
        logger.exception(f"Unexpected error: {e}")

fetch_and_log_runs_structured(config.MLFLOW_DEEPAR_CONFIG['experiment_name'])

2025-07-28 01:20:03.552 | INFO     | __main__:fetch_and_log_runs_structured:11 - MLflow tracking URI set to: file:///D:%5CCML%5CTerm%208%5CML%20projects%5Cforecasting_workspace%5Coceanwave_forecast%5Cmlruns
2025-07-28 01:20:03.580 | ERROR    | __main__:fetch_and_log_runs_structured:18 - Experiment 'Oceanwave_DeepAR_Training' not found at D:\CML\Term 8\ML projects\forecasting_workspace\oceanwave_forecast\mlruns.


In [88]:
raw_path   = config.RAW_DATA_DIR / "Standard meteorological data 2024" / "46088h2024.txt"
df_raw     = data_manager.extract_raw_data(raw_path)
df_clean   = data_pipeline.preprocess_ocean_data(df_raw)
# df_clean   = df_clean.loc[config.START_DATE : config.END_DATE]

# split target & features
Y = df_clean[config.TARGETS]
X = df_clean.drop(columns=config.TARGETS)

y_train, y_test, X_train, X_test = temporal_train_test_split(
    y=Y, X=X, test_size=config.HORIZON * 10
)


D:\CML\Term 8\ML projects\forecasting_workspace\oceanwave_forecast\oceanwave_forecast\data_manager.py:43: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(


DataFrame shape: (52650, 13)

Info:
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 52650 entries, 2024-01-01 00:00:00 to 2024-12-31 23:50:00
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   WDIR    52650 non-null  float64
 1   WSPD    52650 non-null  float64
 2   GST     52650 non-null  float64
 3   WVHT    52650 non-null  float64
 4   DPD     52650 non-null  float64
 5   APD     52650 non-null  float64
 6   MWD     52650 non-null  float64
 7   PRES    52650 non-null  float64
 8   ATMP    52650 non-null  float64
 9   WTMP    52650 non-null  float64
 10  DEWP    52650 non-null  float64
 11  VIS     52650 non-null  float64
 12  TIDE    52650 non-null  float64
dtypes: float64(13)
memory usage: 5.6 MB

Descriptive statistics:
               WDIR          WSPD           GST          WVHT           DPD  
count  52650.000000  52650.000000  52650.000000  52650.000000  52650.000000   
mean     194.421026      4.962283      6.2

D:\CML\Term 8\ML projects\forecasting_workspace\oceanwave_forecast\oceanwave_forecast\data_pipeline.py:111: FutureWarning: 'H' is deprecated and will be removed in a future version. Use ``sep='h'`` instead.
  data_ocean_hourly = data_ocean_clean.resample('H').mean()


# 2. FEATURE ENGINEERING FOR DEEPAR


In [89]:
pipe_X, pipe_Y = data_pipeline.get_pipelines(list(X_train.columns))

X_train_transformed = pipe_X.fit_transform(X_train)
X_test_transformed  = pipe_X.transform(X_test)
y_train_transformed = pipe_Y.fit_transform(y_train)
y_test_transformed  = pipe_Y.transform(y_test)


c:\Users\akashv22\AppData\Local\anaconda3\envs\fc_env\lib\site-packages\pmdarima\arima\_validation.py:76: UserWarning: stepwise model cannot be fit in parallel (n_jobs=1). Falling back to stepwise parameter search.
  warnings.warn('stepwise model cannot be fit in parallel (n_jobs=%i). ')
c:\Users\akashv22\AppData\Local\anaconda3\envs\fc_env\lib\site-packages\pmdarima\arima\_validation.py:76: UserWarning: stepwise model cannot be fit in parallel (n_jobs=1). Falling back to stepwise parameter search.
  warnings.warn('stepwise model cannot be fit in parallel (n_jobs=%i). ')
c:\Users\akashv22\AppData\Local\anaconda3\envs\fc_env\lib\site-packages\pmdarima\arima\_validation.py:76: UserWarning: stepwise model cannot be fit in parallel (n_jobs=1). Falling back to stepwise parameter search.
  warnings.warn('stepwise model cannot be fit in parallel (n_jobs=%i). ')


In [91]:
def _add_calendar(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    hr = df.index.hour
    df["month"] = df.index.month
    df["hour"] = hr
    df["hr_sin"] = np.sin(2 * np.pi * hr / 24)
    df["hr_cos"] = np.cos(2 * np.pi * hr / 24)
    return df


def make_long(
    X: pd.DataFrame,
    y: pd.DataFrame,
    series_col: str = config.ID_COLS[0],                #  identifier of each buoy / station
    time_col:  str = "timestamp"               #  DatetimeIndex will be copied here
) -> pd.DataFrame:
    # combine exogenous & targets side‑by‑side
    df = pd.concat([X, y], axis=1)

    df[time_col]   = df.index                  # DatetimeIndex → column
    df[series_col] = X.index.get_level_values(series_col) if isinstance(
        X.index, pd.MultiIndex
    ) else series_col                          # constant string if only one series

    # add calendar features
    df = _add_calendar(df)

    # Adding the time_idx (0,1,2,…) **within each group**
    df[config.ID_COLS[1]] = (
        df.groupby(series_col)[time_col]
          .rank(method="first")
          .astype("int64") - 1
    )
    df = df.drop(columns=[time_col])  # drop the original DatetimeIndex

    return df.reset_index(drop=True)


train_long = make_long(X_train_transformed, y_train_transformed)
test_long  = make_long(X_test_transformed,  y_test_transformed)

print("TRAIN head:\n", train_long.head(3))
print("TEST  head:\n", test_long.head(3))
print("TRAIN shape:\n", train_long.shape)
print("TEST  shape:\n", test_long.shape)

TRAIN head:
        WSPD       GST      PRES      ATMP      WTMP      DEWP  WDIR_sin  \n
0 -0.945944 -0.877987  0.973289 -0.763690 -0.601870 -0.572855  1.712621   \n
1 -1.204516 -1.147758  1.014906 -0.810166 -0.631785 -0.572855  1.633114   \n
2 -1.431427 -1.327605  1.040339 -0.856642 -0.661700 -0.572855  0.121828   \n
\n
   WDIR_cos   MWD_sin   MWD_cos      WVHT       APD  group_id  month  hour  \n
0  0.050028  1.967362  1.453692 -0.767869  0.790348  group_id      1     0   \n
1  0.015358  1.967362  1.453692 -0.706999  1.472553  group_id      1     1   \n
2  0.756579  1.967336  1.453692 -0.783087  1.503329  group_id      1     2   \n
\n
     hr_sin    hr_cos  time_idx  \n
0  0.000000  1.000000         0  \n
1  0.258819  0.965926         1  \n
2  0.500000  0.866025         2  \n
TEST  head:
        WSPD       GST      PRES      ATMP      WTMP      DEWP  WDIR_sin  \n
0 -0.170226 -0.308471  1.151319 -0.577787 -0.586912 -0.523410  0.351806   \n
1 -0.212442 -0.364138  1.162880 -0.548739 -0.

In [92]:

def apply_window_summarizer(
    df,
    summarizer: WindowSummarizer,
    target_cols: list[str],
    fit: bool = True
):
    # Get the target columns from the DataFrame
    df_targets = df[target_cols]
    
    # Apply the summarizer
    if fit:
        df_lagged = summarizer.fit_transform(df_targets)
    else:
        df_lagged = summarizer.transform(df_targets)
    
    # Re-join the new features
    return df.join(df_lagged)



# Configure window summarizer
summarizer = WindowSummarizer(
    lag_feature=config.TARGET_WINDOWSUMMARY_CONFIG,
    target_cols=config.TARGETS,
    n_jobs=1,
)

train_long = apply_window_summarizer(
    train_long,
    summarizer,
    config.TARGETS,
    fit=True
)

test_long = apply_window_summarizer(
    test_long,
    summarizer,
    config.TARGETS,
    fit=False
)

print("TRAIN head:\n", train_long.head(3))
print("TEST  head:\n", test_long.head(3))
print("TRAIN shape:\n", train_long.shape)
print("TEST  shape:\n", test_long.shape)

TRAIN head:
        WSPD       GST      PRES      ATMP      WTMP      DEWP  WDIR_sin  \n
0 -0.945944 -0.877987  0.973289 -0.763690 -0.601870 -0.572855  1.712621   \n
1 -1.204516 -1.147758  1.014906 -0.810166 -0.631785 -0.572855  1.633114   \n
2 -1.431427 -1.327605  1.040339 -0.856642 -0.661700 -0.572855  0.121828   \n
\n
   WDIR_cos   MWD_sin   MWD_cos  ...  WVHT_mean_24_48  APD_lag_1 APD_lag_2  \n
0  0.050028  1.967362  1.453692  ...              NaN        NaN       NaN   \n
1  0.015358  1.967362  1.453692  ...              NaN   0.790348       NaN   \n
2  0.756579  1.967336  1.453692  ...              NaN   1.472553  0.790348   \n
\n
   APD_lag_3  APD_lag_4  APD_lag_24  APD_lag_48  APD_lag_72  APD_mean_1_24  \n
0        NaN        NaN         NaN         NaN         NaN            NaN   \n
1        NaN        NaN         NaN         NaN         NaN            NaN   \n
2        NaN        NaN         NaN         NaN         NaN            NaN   \n
\n
   APD_mean_24_48  \n
0          

In [93]:
print(train_long.dtypes.to_frame(name="Data Type"))


                Data Type
WSPD              float64
GST               float64
PRES              float64
ATMP              float64
WTMP              float64
DEWP              float64
WDIR_sin          float64
WDIR_cos          float64
MWD_sin           float64
MWD_cos           float64
WVHT              float64
APD               float64
group_id           object
month               int32
hour                int32
hr_sin            float64
hr_cos            float64
time_idx            int64
WVHT_lag_1        float64
WVHT_lag_2        float64
WVHT_lag_3        float64
WVHT_lag_4        float64
WVHT_lag_24       float64
WVHT_lag_48       float64
WVHT_lag_72       float64
WVHT_mean_1_24    float64
WVHT_mean_24_48   float64
APD_lag_1         float64
APD_lag_2         float64
APD_lag_3         float64
APD_lag_4         float64
APD_lag_24        float64
APD_lag_48        float64
APD_lag_72        float64
APD_mean_1_24     float64
APD_mean_24_48    float64


In [94]:

# build the exclude list
exclude = set(config.TARGETS + config.ID_COLS)

# all other columns become covariates
covariate_variables = [col for col in train_long.columns if col not in exclude]

print("Covariates:", covariate_variables)

Covariates: ['WSPD', 'GST', 'PRES', 'ATMP', 'WTMP', 'DEWP', 'WDIR_sin', 'WDIR_cos', 'MWD_sin', 'MWD_cos', 'month', 'hour', 'hr_sin', 'hr_cos', 'WVHT_lag_1', 'WVHT_lag_2', 'WVHT_lag_3', 'WVHT_lag_4', 'WVHT_lag_24', 'WVHT_lag_48', 'WVHT_lag_72', 'WVHT_mean_1_24', 'WVHT_mean_24_48', 'APD_lag_1', 'APD_lag_2', 'APD_lag_3', 'APD_lag_4', 'APD_lag_24', 'APD_lag_48', 'APD_lag_72', 'APD_mean_1_24', 'APD_mean_24_48']


In [95]:
train_long_clean = train_long.dropna(subset=covariate_variables).reset_index(drop=True)
test_long_clean  = test_long.dropna(subset=covariate_variables).reset_index(drop=True)

# 3. TIMESERIESDATASET CONFIGURATION


In [96]:
config.TARGETS

['WVHT', 'APD']

In [97]:
# 4. build the training TimeSeriesDataSet
train_ds = TimeSeriesDataSet(
    data=train_long_clean,
    time_idx=config.ID_COLS[1],
    target=config.TARGETS,
    group_ids=[config.ID_COLS[0]], 
    max_encoder_length=config.WINDOW,
    max_prediction_length=config.HORIZON,
    static_reals=None,
    time_varying_known_categoricals=None,
    time_varying_known_reals=covariate_variables,
    time_varying_unknown_categoricals=None,
    time_varying_unknown_reals=config.TARGETS,
    target_normalizer=MultiNormalizer(
        [GroupNormalizer(groups=[config.ID_COLS[0]]) for _ in config.TARGETS]
    ),
    add_relative_time_idx=True,
    add_target_scales=True,
    add_encoder_length=True
)

test_ds = TimeSeriesDataSet.from_dataset(
    train_ds, 
    test_long_clean, 
    stop_randomization=True
)



In [98]:
train_loader = train_ds.to_dataloader(train=True, batch_size=config.BATCH_SIZE, num_workers=config.NUM_WORKERS)
test_loader = test_ds.to_dataloader(train=False, batch_size=config.BATCH_SIZE, num_workers=config.NUM_WORKERS)

In [99]:
plotting.plot_dataloader_sample(train_loader, config.TESTING_REPORTS_DIR / "Testing_data_sample.png")

D:\CML\Term 8\ML projects\forecasting_workspace\oceanwave_forecast\oceanwave_forecast\plotting.py:112: UserWarning: The figure layout has changed to tight
  plt.tight_layout()


Sample batch data visualization saved to D:\CML\Term 8\ML projects\forecasting_workspace\oceanwave_forecast\reports\testing\Testing_data_sample.png


# 4. TRAINING AND EVALUATION WITH MLFLOW


In [103]:
from lightning.pytorch.callbacks import ModelCheckpoint

def run_deepar_training(
    train_loader,
    test_loader,
    train_ds,
    run_number: int,
    model_name: str = "DeepAR",
):
    """
    Wraps the DeepAR training and evaluation in an MLflow experiment.
    """
    exp_manager = mlflow_utils.MLflowExperimentManager(
        experiment_name=config.MLFLOW_DEEPAR_CONFIG['experiment_name'],
        run_number=run_number,
        tags=config.MLFLOW_DEEPAR_CONFIG['tags']
    )

    run = exp_manager.start_mlflow_run(run_name_prefix=model_name)

    if run.info.status == "FINISHED":
        print(f"{model_name} run already complete (id={run.info.run_id}) – skipping.")
        return run

    try:
        # Log parameters
        exp_manager.log_params(config.DEEPAR_CONFIG)
        exp_manager.log_param("model_type", model_name)
        exp_manager.log_param("targets", ",".join(config.TARGETS))
        exp_manager.log_param("batch_size", config.BATCH_SIZE)
        exp_manager.log_param("max_epochs", config.MAX_EPOCHS)
        exp_manager.log_param("learning_rate", config.LEARNING_RATE)
        exp_manager.log_param("gradient_clip_val", config.GRADIENT_CLIP_VAL)
        exp_manager.log_param("optimizer", config.OPTIMIZER)
        exp_manager.log_param("window", config.WINDOW)
        exp_manager.log_param("horizon", config.HORIZON)

        # Setup callbacks
        early_stop = EarlyStopping(
            monitor="val_loss",
            patience=config.EARLY_STOP_PATIENCE,
            min_delta=config.EARLY_STOP_MIN_DELTA,
            mode="min",
        )
        lr_logger = LearningRateMonitor(logging_interval="epoch")
        checkpoint = ModelCheckpoint(
            monitor="val_loss",
            mode="min",
            save_top_k=1,
            filename=f"{model_name}-{{epoch:02d}}-val_loss={{val_loss:.4f}}"
        )
        print_metrics = training.PrintMetricsCallback(print_every_n_epochs=10)

        # Setup trainer
        trainer = Trainer(
            max_epochs=config.MAX_EPOCHS,
            accelerator="auto",
            gradient_clip_val=config.GRADIENT_CLIP_VAL,
            callbacks=[lr_logger, early_stop, checkpoint, print_metrics],
            limit_train_batches=config.LIMIT_TRAIN_BATCHES,
            val_check_interval=config.VAL_CHECK_INTERVAL,
            log_every_n_steps=config.LOG_EVERY_N_STEPS,
            enable_progress_bar=True,
        )

        # Build multivariate loss
        multivar_loss = MultiLoss(
            [NormalDistributionLoss() for _ in config.TARGETS]
        )

        # Instantiate model
        model = DeepAR.from_dataset(
            train_ds,
            learning_rate=config.LEARNING_RATE,
            hidden_size=config.DEEPAR_CONFIG["lstm_hidden_dim"],
            rnn_layers=config.DEEPAR_CONFIG["lstm_layers"],
            dropout=config.DEEPAR_CONFIG["lstm_dropout"],
            optimizer=config.OPTIMIZER,
            loss=multivar_loss,
        )

        print(f"Training {model_name}…")
        trainer.fit(model, train_loader, test_loader)

        # Load best checkpoint
        best_model_path = checkpoint.best_model_path
        exp_manager.log_artifact(best_model_path, "model")
        best_model = DeepAR.load_from_checkpoint(best_model_path)

        print("Evaluating model…")
        predictions = best_model.predict(
            test_loader,
            trainer_kwargs=dict(accelerator="auto"),
            batch_size=1
        )

        # Calculate and log metrics
        metrics = {
            'MAE': MAE(),
            'RMSE': RMSE(),
            'SMAPE': SMAPE(),
            'MAPE': MAPE()
        }
        eval_metrics = training.model_evaluation(predictions, metrics)
        exp_manager.log_metrics(eval_metrics)
        plotting.model_plotting_function(best_model,test_loader)

        print(f"✅ {model_name} training complete.")

    except Exception as e:
        print(f"❌ Error in {model_name}: {e}")
        raise

    finally:
        exp_manager.end_mlflow_run()

    return run


In [ ]:
run_deepar_training(
    train_loader=train_loader,
    test_loader=test_loader,
    train_ds=train_ds,
    run_number=20
)

2025-07-28 01:42:02.713 | INFO     | oceanwave_forecast.mlflow_utils:_set_tracking_uri:55 - MLflow tracking URI set to: file:///D:%5CCML%5CTerm%208%5CML%20projects%5Cforecasting_workspace%5Coceanwave_forecast%5Cmlruns
2025-07-28 01:42:02.722 | INFO     | oceanwave_forecast.mlflow_utils:get_or_create_experiment:81 - Found existing experiment 'Oceanwave_DeepAR_Training' with ID 828880404737217571.
2025-07-28 01:42:02.775 | INFO     | oceanwave_forecast.mlflow_utils:start_mlflow_run:134 - Started MLflow run with ID: 57f8ee64fef04f62b8090d1c05ea5e31
2025-07-28 01:42:02.782 | INFO     | oceanwave_forecast.mlflow_utils:log_params:159 - Logged parameters: dict_keys(['lstm_hidden_dim', 'lstm_layers', 'lstm_dropout', 'embedding_dim', 'num_class'])
2025-07-28 01:42:02.785 | INFO     | oceanwave_forecast.mlflow_utils:log_param:170 - Logged parameter: model_type=DeepAR
2025-07-28 01:42:02.785 | INFO     | oceanwave_forecast.mlflow_utils:log_param:170 - Logged parameter: targets=WVHT,APD
2025-07-28

Training DeepAR…


Sanity Checking: 0it [00:00, ?it/s]

c:\Users\akashv22\AppData\Local\anaconda3\envs\fc_env\lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:442: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 20 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(



Epoch 0 Summary:
------------------------------


...

2025-07-28 01:43:06.389 | INFO     | oceanwave_forecast.mlflow_utils:log_artifact:205 - Logged artifact: d:\CML\Term 8\ML projects\forecasting_workspace\oceanwave_forecast\notebooks\lightning_logs\version_17\checkpoints\DeepAR-epoch=05-val_loss=val_loss=0.3238.ckpt to model
c:\Users\akashv22\AppData\Local\anaconda3\envs\fc_env\lib\site-packages\lightning\pytorch\utilities\parsing.py:197: UserWarning: Attribute 'loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss'])`.
  rank_zero_warn(
c:\Users\akashv22\AppData\Local\anaconda3\envs\fc_env\lib\site-packages\lightning\pytorch\utilities\parsing.py:197: UserWarning: Attribute 'logging_metrics' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['logging_metrics'])`.
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False


Epoch 10 Summary:
  -> Train Loss: -0.6771
  -> Validation Loss: 0.6619
------------------------------
Evaluating model…


c:\Users\akashv22\AppData\Local\anaconda3\envs\fc_env\lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:442: PossibleUserWarning: The dataloader, predict_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 20 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
2025-07-28 01:43:08.685 | INFO     | oceanwave_forecast.mlflow_utils:end_mlflow_run:147 - Ended MLflow run with ID: 57f8ee64fef04f62b8090d1c05ea5e31


✅ DeepAR training complete.


<ActiveRun: >